# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

## Follow the Directions in Bold. If you get stuck, check out the solutions lecture.

### THE DATA

** Read in the census_data.csv data with pandas**

In [1]:
import pandas as pd

In [2]:
census = pd.read_csv('census_data.csv')

In [3]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [4]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [6]:
def label_fix(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [7]:
census['income_bracket'] = census['income_bracket'].apply(label_fix)

### Perform a Train Test Split on the Data

In [9]:
from sklearn.model_selection import train_test_split

In [11]:
x_data = census.drop('income_bracket', axis = 1)
y_labels = census['income_bracket']
X_train, X_test, y_train, y_test = train_test_split(x_data,y_labels,test_size=0.3,random_state=101)

** Import Tensorflow **

In [12]:
import tensorflow as tf

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

In [14]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ["Female", "Male"])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size=1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [15]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

** Create the continuous feature_columns for the continuous values using numeric_column **

** Put all these variables into a single list with the variable name feat_cols **

In [16]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
            age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [17]:
input_func = tf.estimator.inputs.pandas_input_fn(x = X_train, y = y_train, batch_size=100, num_epochs = None, shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [18]:
model = tf.estimator.LinearClassifier(feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_every_n_hours': 10000, '_tf_random_seed': 1, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_model_dir': '/var/folders/zz/zyxvpxvq6csfxvn_n0002d4m000l95/T/tmp0id2aqii', '_log_step_count_steps': 100, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_session_config': None}


** Train your model on the data, for at least 5000 steps. **

In [31]:
model.train(input_fn = input_func, steps = 10000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /var/folders/zz/zyxvpxvq6csfxvn_n0002d4m000l95/T/tmp0id2aqii/model.ckpt-5000
INFO:tensorflow:Saving checkpoints for 5001 into /var/folders/zz/zyxvpxvq6csfxvn_n0002d4m000l95/T/tmp0id2aqii/model.ckpt.
INFO:tensorflow:step = 5001, loss = 29.0375
INFO:tensorflow:global_step/sec: 178.354
INFO:tensorflow:step = 5101, loss = 34.2161 (0.562 sec)
INFO:tensorflow:global_step/sec: 190.196
INFO:tensorflow:step = 5201, loss = 50.578 (0.526 sec)
INFO:tensorflow:global_step/sec: 178.45
INFO:tensorflow:step = 5301, loss = 46.5775 (0.561 sec)
INFO:tensorflow:global_step/sec: 180.006
INFO:tensorflow:step = 5401, loss = 98.4301 (0.555 sec)
INFO:tensorflow:global_step/sec: 197.312
INFO:tensorflow:step = 5501, loss = 168.064 (0.507 sec)
INFO:tensorflow:global_step/sec: 190.603
INFO:tensorflow:step = 5601, loss = 198.014 (0.525 sec)
INFO:tensorflow:global_step/sec: 192.272
INFO:tensorflow:step = 5701, loss = 107.71 (0.520 

INFO:tensorflow:global_step/sec: 199.868
INFO:tensorflow:step = 13201, loss = 31.5616 (0.500 sec)
INFO:tensorflow:global_step/sec: 202.608
INFO:tensorflow:step = 13301, loss = 59.5757 (0.494 sec)
INFO:tensorflow:global_step/sec: 199.974
INFO:tensorflow:step = 13401, loss = 33.9873 (0.500 sec)
INFO:tensorflow:global_step/sec: 199.252
INFO:tensorflow:step = 13501, loss = 28.8344 (0.502 sec)
INFO:tensorflow:global_step/sec: 195.273
INFO:tensorflow:step = 13601, loss = 34.9857 (0.512 sec)
INFO:tensorflow:global_step/sec: 196.845
INFO:tensorflow:step = 13701, loss = 46.9045 (0.508 sec)
INFO:tensorflow:global_step/sec: 200.228
INFO:tensorflow:step = 13801, loss = 41.2131 (0.499 sec)
INFO:tensorflow:global_step/sec: 199.974
INFO:tensorflow:step = 13901, loss = 34.5866 (0.500 sec)
INFO:tensorflow:global_step/sec: 200.517
INFO:tensorflow:step = 14001, loss = 27.1783 (0.499 sec)
INFO:tensorflow:global_step/sec: 201.599
INFO:tensorflow:step = 14101, loss = 52.4107 (0.496 sec)
INFO:tensorflow:glob

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [34]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x = X_test, batch_size = len(X_test), shuffle = False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [35]:
predictions = list(model.predict(pred_fn))

INFO:tensorflow:Restoring parameters from /var/folders/zz/zyxvpxvq6csfxvn_n0002d4m000l95/T/tmp0id2aqii/model.ckpt-15000


** Each item in your list will look like this: **

In [36]:
predictions[0]

{'class_ids': array([0]),
 'classes': array([b'0'], dtype=object),
 'logistic': array([ 0.20098951], dtype=float32),
 'logits': array([-1.38012123], dtype=float32),
 'probabilities': array([ 0.79901046,  0.20098953], dtype=float32)}

** Create a list of only the class_ids key values from the prediction list of dictionaries, these are the predictions you will use to compare against the real y_test values. **

In [37]:
final_preds = []
for pred in predictions:
    final_preds.append(pred['class_ids'][0])

In [38]:
final_preds[:10]

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [39]:
from sklearn.metrics import classification_report

In [40]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.85      0.94      0.89      7436
          1       0.69      0.46      0.55      2333

avg / total       0.81      0.82      0.81      9769



# Great Job!